# Visual SLAM

1. Initialization: At first frame, initialize map with 3D points from stereo.
2. Tracking:
   - at frame i+1, match keypoints between i and i+1


In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm

from lac.perception.vision import LightGlueMatcher
from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses
from lac.util import load_data
import lac.params as params

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(
    os.path.expanduser("~/LunarAutonomyChallenge/output/NavAgent/map1_preset4_gtnav_steer")
)
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

map = np.load("../../data/heightmaps/competition/Moon_Map_01_preset_0.dat", allow_pickle=True)

In [ ]:
# Load the images

left_imgs = {}
right_imgs = {}

for img_name in os.listdir(data_path / "FrontLeft"):
    left_imgs[int(img_name.split(".")[0])] = cv2.imread(
        str(data_path / "FrontLeft" / img_name), cv2.IMREAD_GRAYSCALE
    )

for img_name in os.listdir(data_path / "FrontRight"):
    right_imgs[int(img_name.split(".")[0])] = cv2.imread(
        str(data_path / "FrontRight" / img_name), cv2.IMREAD_GRAYSCALE
    )

assert len(left_imgs.keys()) == len(right_imgs.keys())
img_idxs = sorted(left_imgs.keys())

In [ ]:
fig = plot_surface(map)
fig = plot_poses(poses[::20], fig=fig)
fig.show()

In [ ]:
from lightglue import LightGlue, SuperPoint

In [ ]:
matcher = LightGlueMatcher()

feats0, feats1, matches01 = matcher.match(left_imgs[2], right_imgs[2])
matches = matches01["matches"]  # indices with shape (K,2)
points0 = feats0["keypoints"][matches[..., 0]]  # coordinates in image #0, shape (K,2)
points1 = feats1["keypoints"][matches[..., 1]]  # coordinates in image #1, shape (K,2)

disparities = (points0 - points1)[:, 0]
depths = params.FL_X * params.STEREO_BASELINE / disparities

In [ ]:
feats0

In [ ]:
def stereo_to_rover_points(left_img, right_img):
    feats0, feats1, matches01 = matcher.match(left_img, right_img)
    matches = matches01["matches"]  # indices with shape (K,2)
    points0 = feats0["keypoints"][matches[..., 0]]  # coordinates in image #0, shape (K,2)
    points1 = feats1["keypoints"][matches[..., 1]]  # coordinates in image #1, shape (K,2)

    disparities = (points0 - points1)[:, 0]
    depths = params.FL_X * params.STEREO_BASELINE / disparities

    points_rover = []
    for pixel, depth in zip(points0, depths):
        point_rover = project_pixel_to_rover(pixel, depth, "FrontLeft", cam_config)
        points_rover.append(point_rover)
    return np.array(points_rover)

In [ ]:
all_points_world = []

for i in np.arange(100, 2000, 100):
    points_rover = stereo_to_rover_points(left_imgs[i], right_imgs[i])
    points_world = apply_transform(poses[i], points_rover)
    all_points_world.append(points_world)

all_points_world = np.concatenate(all_points_world, axis=0)

In [ ]:
fig = plot_surface(map)
# fig = plot_poses(poses[::20], fig=fig)
fig = plot_3d_points(all_points_world, fig=fig)
fig.show()

In [ ]:
from dataclasses import dataclass


@dataclass
class MapPoint:
    xyz: np.ndarray
    descriptor: np.ndarray
    label: str

In [ ]:
class SLAM:
    def __init__(self):
        self.matcher = LightGlueMatcher()